In [ ]:
# Imports
from submission_utils import AutoSubmitter
from model_utils import *
from Preprocessing import FeatureEngineerer

# Load preprocessed data
merged_hornsea = pd.read_parquet("data/preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("data/preprocessed_pes_with_energy.parquet")

# Set up FeatureEngineerer instances
feature_engineerer_wind = FeatureEngineerer(label = 'Wind_MWh_credit', columns_to_ohe=['unavailabilityType', 'affectedUnit'])
feature_engineerer_solar = FeatureEngineerer(label = "Solar_MWh_credit", columns_to_ohe=[])

# Perform feature engineering
feature_engineerer_wind.perform_feature_engineering(merged_hornsea, deployment = False)
feature_engineerer_solar.perform_feature_engineering(merged_pes, deployment = False)

# Set up model instances
wind_model = LGBMRegressorModel(feature_engineerer_wind, quantiles=np.round(np.arange(0.1, 1.0, 0.1), 1), model_save_dir="Models/lgbm_model_wind", load_pretrained=True)
solar_model = LGBMRegressorModel(feature_engineerer_solar, quantiles=np.round(np.arange(0.1, 1.0, 0.1), 1), model_save_dir="Models/lgbm_model_solar", load_pretrained=True)

# Set up AutoSubmitter instance
submitter = AutoSubmitter(wind_model, solar_model)
# Create submission data
submitter.fetch_data().prepare_data().predict().prepare_submission()

In [ ]:
# Check results before submisson
submitter.predictions["final"]

In [ ]:
# Submit results
submitter.submit()